# Cynthia Kielpinski

## Research Interests:

By majoring in statistics, I have always had a passion for the process behind statistics. The collection of data, the wrangling and processing of the data, exploratory analysis of the data, interpreting the analysis, and then finalizing the process by formulating a conclusion from the analysis. One fundamental ideal in statistics I have learned is that without an interdiscplinary field, you do not have the data and you are simply performing pure statistics. As per this comment, I have never found interest in pure statistics thus I found a passion in the interdisciplinary field of ecology and environmental sciences. For this project, my research interests are focus on exploratory statistics (required) and an environmental or ecological factor. Environmental data is more prominent  than ecological from my persepctive thus I have decided to focus on an environmental influence. Since our group decided on usng a summer and winter Olympics dataset, I though to refine my first research question to involve different countries performance in the summer and winter olympics influenced by their "local" climate. Competition is always a prfound topic, and I was also interested in how countries perform in their home country versus a visiting country. 

## Research Question: 

1. **Does the climate zone of a country denoted by the Köppen climate classification system determine a correlation between the probability or likelihood of preforming more succesfully in the summer or winter Olmypics?** For example, does Canada have a better probability of winning medals in the winter Olympics since it is in group D of the Köppen climate classification system (continental climate) defined specifically as subartic climate? Does Brazil have a better probability of winning medals in the summer Olympics since it is in group A of the Köppen climate classification system (tropical climates) defined as a tropical wet or savannah climate? Or is there no correlation between a countries climate classification and which Olympics the country preforms better in? I will explore the correlations between countries and their climates in regards to number of medals won in each of the respective summer or winter Olympic events. 

## Exploratory Data Analysis Plan: 

To determine some answers into my research questions, I will use descriptive and exploratory techniques. Some of my ideas for more advanced analyses are: regression analysis, principal component analysis, analysis of variance and other simple but profound statistical methods. From briefly looking over the datasets, the above methods are a brief thought to the possibilities when exploring the data. After wrangling the data, I will define the specific analyses methods I choose to use. I have a background in Statistics, thus deciding what analyses will be used at a later stage will be an simplistic decision. 

I will complete as little wrangling as needed, but I have visioned the data to have a few more columns in regards to location, hosting city, average number of medals won by country, total number of medals won, total number of bronze medals, total number of silver medals, total number of gold medals, and anything data-wise that can be combined from the other datasets we have got approved. Additionally, I think I will merge the "four" datasets, and then remove the columns that aren't needed and remove the missing values if there are any present. Lastly, I think I will also rename the column names to make them easier for analysis.

## Import Packages: (all packages needed for analysis, updated 02-12-2022)

In [6]:
## Import Packages:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
## Data Upload:
OlympicHost = pd.read_csv("../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_hosts.csv")

OlympicMedals = pd.read_csv("../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_medals.csv")

GDPCountry = pd.read_csv("../../data/raw/GDPbyCountry.csv")

HistTemp = pd.read_csv("../../data/raw/HistoricalTemperatureData.csv")


#### Notes: About the Köppen Climate Classification System and the GDP Datasets

[Köppen Climate Classification System Data:](https://tcktcktck.org/countries)

[Köppen Climate Classification System Reference Guide:](https://en.wikipedia.org/wiki/K%C3%B6ppen_climate_classification#:~:text=The%20K%C3%B6ppen%20climate%20classification%20divides,is%20represented%20by%20a%20letter.)

The license for the first weblink is not listed, and I cannot find any license regarding this information. I may alter my research to use [average temperature dataset per country](https://datacatalog.worldbank.org/search/dataset/00402760) in reference to the wikipedia page to determine some way of classifying climate. This link is from the World Bank Website which has the following license as described below. 

[Gross Domestic Product Data:](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD)

By looking into the licenses for the data from the [World Bank Website](https://www.worldbank.org/en/about/legal/terms-of-use-for-datasets) it quotes that the license "[u]nless specifically labeled otherwise, these Datasets are provided to you under a Creative Commons Attribution 4.0 International License (CC BY 4.0), with the additional terms below."


## Initial Observations: 

In [8]:
OlympicHost.head(n=5)

,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014


In [9]:
OlympicMedals.head(n=5)

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,Italy,IT,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,Italy,IT,ITA
2,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,Norway,NO,NOR
3,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,Norway,NO,NOR
4,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,Sweden,SE,SWE


In [10]:
## This data set is going to need a bit of wrangling...
GDPCountry.head(5)

,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Last Updated Date,2022-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2.012000e+03,2.013000e+03,2.014000e+03,2.015000e+03,2.016000e+03,2.017000e+03,2.018000e+03,2.019000e+03,2.020000e+03,2.021000e+03
4,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.615084e+09,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.368970e+09,2.610039e+09,3.126019e+09


In [11]:
## Will have to compare the 3 letter country codes with the medals csv...
HistTemp.head(5)

,ISO_3DIGIT,Jan_Temp,Feb_temp,Mar_temp,Apr_Temp,May_temp,Jun_Temp,July_Temp,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp
0,AFG,0.07,2.11,7.60,13.37,18.22,23.20,25.26,23.77,19.03,12.99,7.00,2.43,12.92
1,AGO,22.58,22.68,22.78,22.35,20.74,18.37,17.95,19.90,22.19,23.18,22.79,22.61,21.51
2,ALB,2.02,3.22,6.04,9.92,14.44,17.93,20.54,20.48,17.16,12.27,7.58,3.65,11.27
3,ARE,18.43,19.43,22.61,26.58,30.62,32.46,33.80,33.55,31.74,28.34,24.06,20.28,26.83
4,ARG,20.80,19.90,17.51,14.05,10.65,7.66,7.42,9.02,11.53,14.67,17.54,19.83,14.22
